#pytorch模型转换为onnx模型

In [2]:
#导入模块
import torch
import torch.onnx
from net import SimCLRStage2

In [4]:
#需要调整的有1.权重文件的路径；2.onnx模型的输出路径及名称
#加载模型，simclrv2.pth替换为实际路径
model = SimCLRStage2(num_class=10)
model.eval()
checkpoint = torch.load(r"torch\simclrv2.pth", map_location="cpu")
model.load_state_dict(checkpoint)
#输入
input = torch.randn(1, 3, 32, 32, requires_grad=True)
#转换onnx，simclrv2_bm1684x.onnx输出onnx模型名称
torch.onnx.export(model,
        input,
        'simclrv2_bm1684x.onnx', # name of the exported onnx model
        opset_version=15,
        export_params=True,
        do_constant_folding=True)

#测试不同模型的精度

In [5]:
#测试pytorch模型
import torch
from torchvision.datasets import CIFAR10
from torchvision import transforms
import net

In [ ]:
#需要调整的有1.测试数据下载的路径；2.batch_size可根据需要修改；3.加载的权重文件路径
if torch.cuda.is_available():
        DEVICE = torch.device("cuda:" + str(0))
        torch.backends.cudnn.benchmark = True
else:
        DEVICE = torch.device("cpu")
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
#替换自己的下载路径
eval_dataset=CIFAR10(root='dataset', train=False, transform=transforms, download=True)
eval_data=torch.utils.data.DataLoader(eval_dataset,batch_size=1, shuffle=False, num_workers=16, )
model=net.SimCLRStage2(num_class=10).to(DEVICE)
#替换实际的权重文件路径
model.load_state_dict(torch.load(r"torch\simclrv2.pth", map_location='cpu'), strict=False)
total_correct_1, total_correct_5, total_num = 0.0, 0.0, 0.0
model.eval()
with torch.no_grad():
        print("batch", " "*1, "top1 acc", " "*1,"top5 acc" )
        for batch, (data, target) in enumerate(eval_data):
            data, target = data.to(DEVICE) ,target.to(DEVICE)
            pred=model(data)
            data_size = len(eval_data)
            print("eval_data的数据集大小为:", data_size)
            total_num += data.size(0)
            prediction = torch.argsort(pred, dim=-1, descending=True)
            top1_acc = torch.sum((prediction[:, 0:1] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            top5_acc = torch.sum((prediction[:, 0:5] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            total_correct_1 += top1_acc
            total_correct_5 += top5_acc
            print("  {:02}  ".format(batch+1)," {:02.3f}%  ".format(top1_acc / data.size(0) * 100),"{:02.3f}%  ".format(top5_acc / data.size(0) * 100))
print("all eval dataset:","top1 acc: {:02.3f}%".format(total_correct_1 / total_num * 100), "top5 acc:{:02.3f}%".format(total_correct_5 / total_num * 100))

In [ ]:
#测试onnx模型精度
import torch
import torchvision
import onnxruntime
from torchvision import transforms
import numpy as np

In [ ]:
#需要调整的有1.测试数据下载的路径；2.batch_size可根据需要修改；3.加载的onnx模型的路径
# 加载 CIFAR-10 测试集
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
testset = torchvision.datasets.CIFAR10(root='dataset', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=16)

# 加载 ONNX 模型，需要替换onnx的实际路径
onnx_model_path = 'simclrv2_bm1684x.onnx'
ort_session = onnxruntime.InferenceSession(onnx_model_path)

# 获取输入和输出名称
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name

# 对测试集进行推理
correct = 0
total = 0

for data in testloader:
        images, labels = data
        images_np = images.numpy()
        outputs = ort_session.run([output_name], {input_name: images_np})
        _, predicted = torch.max(torch.from_numpy(outputs[0]), 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('准确率：%02.3f %%' % (100 * correct / total))

In [ ]:
#测试bmodel的精度
import pickle
import numpy as np
import json

In [ ]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
#需要调整的有1.测试数据集的路径；2.分类结果json文件的路径
#获取标签
test_dict = unpickle(r'dataset\test_batch')
testlabels = np.array(test_dict[b'labels'])

# 从 JSON 文件中读取预测结果
with open(r'model_simclr_1b_f16.bmodel_test_batch_opencv_python_result.json', 'r') as file:
    data = json.load(file)
true_img = 0
total = 0
i = 0
for prediction,label in zip(data,testlabels):
    actual_label = label
    predicted_label = prediction['prediction']
    if actual_label == predicted_label:
        true_img = true_img + 1
    total = total+1
acc = true_img / total
print('准确率：%02.3f %%' % (100.0 * true_img / total))